In [1]:
def _calc_output_size(input_size, padding, dilation, kernel_size, stride):
    input_size_padded = input_size + 2 * padding
    kernel_dilated = (kernel_size - 1) * (dilation - 1) + kernel_size
    output_size = (input_size_padded - kernel_dilated) // stride + 1
    return output_size

In [17]:

# layer 1
in_features = 64
output_size = _calc_output_size(in_features, padding=3, dilation=1, kernel_size=7, stride=2)
print('conv1\t\t',in_features, output_size,'padding=3, dilation=1, kernel_size=7, stride=2')

# maxpool
in_features = output_size
output_size = _calc_output_size(in_features, padding=1, dilation=1, kernel_size=3, stride=2)
print('maxpool\t\t',in_features, output_size,'padding=0, dilation=1, kernel_size=3, stride=2')

# layer 2
in_features = output_size
output_size = _calc_output_size(in_features, padding=1, dilation=1, kernel_size=3, stride=2)
print('conv2..1\t',in_features, output_size, 'padding=1, dilation=1, kernel_size=3, stride=2')

# layer 3
in_features = output_size
output_size = _calc_output_size(in_features, padding=1, dilation=1, kernel_size=3, stride=1)
print('conv2..2\t',in_features, output_size, 'padding=1, dilation=1, kernel_size=3, stride=1')

# layer 4
in_features = output_size
output_size = _calc_output_size(in_features, padding=1, dilation=1, kernel_size=3, stride=1)
print('conv2..3\t',in_features, output_size,'padding=1, dilation=1, kernel_size=3, stride=1')

# layer 5
in_features = output_size
output_size = _calc_output_size(in_features, padding=1, dilation=1, kernel_size=3, stride=1)
print('conv2..4\t',in_features, output_size,'padding=1, dilation=1, kernel_size=3, stride=1')

# layer 6


conv1		 64 32 padding=3, dilation=1, kernel_size=7, stride=2
maxpool		 32 16 padding=0, dilation=1, kernel_size=3, stride=2
conv2..1	 16 8 padding=1, dilation=1, kernel_size=3, stride=2
conv2..2	 8 8 padding=1, dilation=1, kernel_size=3, stride=1
conv2..3	 8 8 padding=1, dilation=1, kernel_size=3, stride=1
conv2..4	 8 8 padding=1, dilation=1, kernel_size=3, stride=1


In [18]:

import torch.nn as nn

In [61]:
class ResidualBlock(nn.Module):

    def __init__(self, in_channels, out_channels, stride=1, expansion=1,downsampling=1):
        super().__init__()
        
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.expansion = expansion
        self.downsampling = downsampling
        self.expanded_channels = self.out_channels * self.expansion
        self.kernel_size = 3

        self.blocks = nn.Sequential(
            
#             nn.Identity())
        
            # first conv layer
            nn.Conv2d(self.in_channels, self.out_channels, kernel_size=self.kernel_size, stride=self.downsampling, padding=self.kernel_size//2, bias=False),
            nn.BatchNorm2d(self.out_channels),
            nn.ReLU(inplace=True),

            # second conv layer
            nn.Conv2d(self.out_channels, self.expanded_channels, kernel_size=self.kernel_size, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(self.out_channels))

        # shortcut
        if self.in_channels == self.out_channels:
            self.shortcut = nn.Identity()
        else:
#             self.shortcut = nn.Identity()
            self.shortcut = nn.Sequential(
                nn.Conv2d(self.in_channels, self.expanded_channels, kernel_size=1, stride=self.downsampling, bias=False)
                # include BatchNorm2d?
            )
        
        self.activate = nn.ReLU(inplace=True)

    def forward(self, x):
        # blocks
        out = self.blocks(x)

        shortcut = self.shortcut(x)
        
        print(out.shape)
        print(shortcut.shape)

        out = self.activate(out + shortcut)

        return out

In [62]:
block = ResidualBlock(32,64)
block


ResidualBlock(
  (blocks): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (shortcut): Sequential(
    (0): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
  )
  (activate): ReLU(inplace=True)
)

In [63]:
dummy = torch.ones([1,32,224,224])
block(dummy).shape

torch.Size([1, 64, 222, 222])
torch.Size([1, 64, 224, 224])


RuntimeError: The size of tensor a (222) must match the size of tensor b (224) at non-singleton dimension 3